In [1]:
from src.snomedFS import determine_best_concepts

In [3]:
# Specify file paths for the required data
child_parents_path = 'data/knowledge-graph-child-parent.csv'
instance_data_path = 'data/instance-data.csv'
cohort_path = 'data/cohorts.csv'

# Initialise the feature selector
feature_selector = determine_best_concepts(child_parents_path = child_parents_path,
                                           instance_data_path = instance_data_path,
                                           groups_path = cohort_path)

# Identify the top 10 SNOMED CT concepts for predicting the class label, which have a minimum difference score of 0.05
min_diff = 0.05
n_features = 10
selected_features = feature_selector.select_concepts(min_diff, n_features)
print('Selected Concepts: ', selected_features)